In [24]:
from elementos_matriz import *
from scipy.optimize import minimize

from qiskit import transpile, QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp

In [25]:
def elementos_matriz(d, a, RA, RB, ZA, ZB):
    # elementos de un cuerpo
    f11 = fpp(1, d, a, RA, RB, ZA, ZB)
    f33 = fpp(3, d, a, RA, RB, ZA, ZB)
    # elementos de dos cuerpos
    g1212 = gpqrs([1,2,1,2], d, a, RA, RB)
    g3434 = gpqrs([3,4,3,4], d, a, RA, RB)
    g1313 = gpqrs([1,3,1,3], d, a, RA, RB)
    g1331 = gpqrs([1,3,3,1], d, a, RA, RB)

    return f11, f33, g1212, g3434, g1313, g1331

In [26]:
def hamiltoniano_H2(d, a, RA, RB, ZA, ZB):
    f11, f33, g1212, g3434, g1313, g1331 = elementos_matriz(d, a, RA, RB, ZA, ZB) # elementos de matriz
    h12 = (2*f11 + g1212 + 2*g1313 - g1331)/4
    h34 = (2*f33 + g3434 + 2*g1313 - g1331)/4
    h0 = f11 + f33 + g1212/4 + g3434/4 + g1313 - g1331/2
    
    hamiltoniano = SparsePauliOp.from_list(
        [
            ("IIII", h0),
            ("IIIZ", -h12),
            ("IIZI", -h12),
            ("IZII", -h34),
            ("ZIII", -h34),
            ("IIZZ", g1212/4),
            ("ZZII", g3434/4),
            ("ZIIZ", g1313/4),
            ("IZZI", g1313/4),
            ("IZIZ", (g1313-g1331)/4),
            ("ZIZI", (g1313-g1331)/4),
            ("YYYY", -g1331/4),
            ("XXYY", -g1331/4),
            ("YYXX", -g1331/4),
            ("XXXX", -g1331/4),
        ]
    )
    return hamiltoniano

In [27]:
# Define the Hamiltonian as a SparsePauliOp
hamiltonian = hamiltoniano_H2(d, a, RA, RB, ZA, ZB)

In [28]:
# Define the ansatz circuit
def ansatz_circuit(params):
    qc = QuantumCircuit(4)
    qc.rx(params[0], 0)
    qc.ry(params[1], 1)
    qc.rx(params[2], 2)
    qc.ry(params[3], 3)
    qc.cx(0, 1)
    qc.cx(2, 3)
    return qc

In [29]:
# Backend for simulation
simulator = AerSimulator()

In [30]:
# Expectation value calculation
def expectation_value(params):
    # Create the ansatz circuit
    qc = ansatz_circuit(params)
    qc.save_statevector()  # Add instruction to save the statevector
    qc = transpile(qc, simulator)
    
    # Simulate the statevector
    result = simulator.run(qc).result()
    statevector = result.data(0)["statevector"].data  # Convert to numpy array
    
    # Compute expectation value
    return np.real(np.vdot(statevector, hamiltonian.to_matrix() @ statevector))

In [31]:
# Optimize the parameters
initial_params = [0.1, 0.1, 0.1, 0.1]
result = minimize(expectation_value, initial_params, method='COBYLA')

In [32]:
# Print results
print("Optimal Parameters:", result.x)
print("Minimum Energy:", result.fun)

Optimal Parameters: [3.14157280e+00 6.37028845e-06 4.73433902e-05 1.26653429e-05]
Minimum Energy: -1.8294595432829657


In [34]:
step = 0.2
distI = 0.5 # distancia interatómica inicial
distF = 4.0 # distancia interatómica final
distancias = np.arange(distI, distF, step)
energias = []
initial_params = [0.1, 0.1, 0.1, 0.1]

for dist in distancias:
    RB = np.array([dist, 0, 0]) # posición núcleo B
    hamiltonian = hamiltoniano_H2(d, a, RA, RB, ZA, ZB)
    result = minimize(expectation_value, initial_params, method='COBYLA')
    energia = result.fun
    energias.append(energia)
    print(
        f"Distancia Interatómica: {np.round(dist, 2)}",
        f"Energía: {energia:.5f}",
    )

Distancia Interatómica: 0.5 Energía: -2.40306
Distancia Interatómica: 0.7 Energía: -2.26524
Distancia Interatómica: 0.9 Energía: -2.12996
Distancia Interatómica: 1.1 Energía: -2.00284
Distancia Interatómica: 1.3 Energía: -1.88515
Distancia Interatómica: 1.5 Energía: -1.77731
Distancia Interatómica: 1.7 Energía: -1.67911
Distancia Interatómica: 1.9 Energía: -1.58971
Distancia Interatómica: 2.1 Energía: -1.50806
Distancia Interatómica: 2.3 Energía: -1.43326
Distancia Interatómica: 2.5 Energía: -1.26255
Distancia Interatómica: 2.7 Energía: -1.25239
Distancia Interatómica: 2.9 Energía: -1.24114
Distancia Interatómica: 3.1 Energía: -1.22932
Distancia Interatómica: 3.3 Energía: -1.21734
Distancia Interatómica: 3.5 Energía: -1.20549
Distancia Interatómica: 3.7 Energía: -1.19397
Distancia Interatómica: 3.9 Energía: -1.18292
